In [ ]:
#!pip uninstall -y torch torchvision torchaudio
#!pip install torch==2.2.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.7/186.7 MB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.9 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a w

In [8]:
!pip install sympy==1.12

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import os
import scipy.sparse as sp
import pickle
import math

# 1. 환경 설정 및 시드 고정
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# =========================================================
# 2. 데이터 로드 (저장된 파일 불러오기)
# =========================================================
base_path = "/content/drive/MyDrive/unstructured/k5_filtered"

# 데이터 읽기
train_df = pd.read_csv(f"{base_path}/train.csv")
val_df   = pd.read_csv(f"{base_path}/val.csv")
test_df  = pd.read_csv(f"{base_path}/test.csv")

# 매핑 정보 로드
with open(f"{base_path}/user2idx.pkl", 'rb') as f: user2idx = pickle.load(f)
with open(f"{base_path}/item2idx.pkl", 'rb') as f: item2idx = pickle.load(f)

n_users = len(user2idx)
n_items = len(item2idx)

print(f"데이터 로드 완료. User: {n_users}, Item: {n_items}")
print(f"전체 Train 데이터 수: {len(train_df)}")

데이터 로드 완료. User: 671, Item: 3485
전체 Train 데이터 수: 88595


In [12]:
# =========================================================
# 3. 데이터 분리 (Positive vs Hard Negative)
# =========================================================
# 1) Positive Data (4점 이상): 그래프 연결 및 정답 학습용
train_pos_df = train_df[train_df['rating'] >= 4.0].copy()

# 2) Hard Negative Data (2점 이하): 오답 학습용
train_neg_df = train_df[train_df['rating'] <= 2.0].copy()

print(f"Positive Interactions (Graph용): {len(train_pos_df)}")
print(f"Hard Negative Interactions: {len(train_neg_df)}")

# 딕셔너리 생성
# Positive: 유저가 좋아하는 아이템 목록
user_pos_items = train_pos_df.groupby("user")["item"].apply(set).to_dict()

# Hard Negative: 유저가 싫어하는 아이템 목록 (샘플링 때 사용)
user_hard_neg_items = train_neg_df.groupby("user")["item"].apply(list).to_dict()

Positive Interactions (Graph용): 46243
Hard Negative Interactions: 11327


In [13]:
# =========================================================
# 4. LightGCN용 인접행렬 생성 (Positive 데이터만 사용!)
# =========================================================
def get_adj_mat(n_users, n_items, pos_df):
    """
    반드시 4점 이상인 pos_df만 넣어서 그래프를 만들어야 함
    """
    user_np = pos_df['user'].values
    item_np = pos_df['item'].values

    R = sp.coo_matrix((np.ones(len(user_np)), (user_np, item_np)), shape=(n_users, n_items))

    top_part = sp.hstack([sp.csr_matrix((n_users, n_users)), R])
    bot_part = sp.hstack([R.T, sp.csr_matrix((n_items, n_items))])
    A = sp.vstack([top_part, bot_part])

    rowsum = np.array(A.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

    norm_adj = d_mat_inv_sqrt.dot(A).dot(d_mat_inv_sqrt).tocoo()
    indices = torch.LongTensor(np.vstack((norm_adj.row, norm_adj.col)))
    values = torch.FloatTensor(norm_adj.data)

    return torch.sparse_coo_tensor(indices, values, torch.Size(norm_adj.shape)).to(device)

print("그래프 생성 중 (Positive Edge Only)...")
Adj_Matrix = get_adj_mat(n_users, n_items, train_pos_df)
print("그래프 생성 완료!")

그래프 생성 중 (Positive Edge Only)...
그래프 생성 완료!


/tmp/ipython-input-1123077487.py:18: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [14]:
# =========================================================
# 5. 모델 정의 (Base LightGCN)
# =========================================================
class LightGCN(nn.Module):
    def __init__(self, n_users, n_items, dim, layers, A_hat):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dim = dim
        self.layers = layers
        self.A_hat = A_hat

        self.user_emb = nn.Embedding(n_users, dim)
        self.item_emb = nn.Embedding(n_items, dim)

        nn.init.normal_(self.user_emb.weight, std=0.1)
        nn.init.normal_(self.item_emb.weight, std=0.1)

    def get_all_embeddings(self):
        users = self.user_emb.weight
        items = self.item_emb.weight
        all_emb = torch.cat([users, items], dim=0)
        embs = [all_emb]

        for _ in range(self.layers):
            all_emb = torch.sparse.mm(self.A_hat, all_emb)
            embs.append(all_emb)

        out = torch.stack(embs, dim=0).mean(dim=0)
        return out[:self.n_users], out[self.n_users:]

In [15]:
# =========================================================
# 6. 샘플링 함수 (Hard Negative 반영)
# =========================================================
def sample_batch_with_hard_neg(batch_size, user_pos_items, user_hard_neg_items, n_items, hard_prob=0.5):
    """
    hard_prob 확률로 Hard Negative(1~2점)를 뽑고, 아니면 Random Negative를 뽑음
    """
    users = np.random.choice(list(user_pos_items.keys()), size=batch_size)
    pos = []
    neg = []

    for u in users:
        # Positive Sampling
        pos.append(np.random.choice(list(user_pos_items[u])))

        # Negative Sampling Strategy
        # 해당 유저에게 Hard Negative(싫어한 영화)가 있고, 확률에 걸리면 그걸 씀
        if (u in user_hard_neg_items) and (len(user_hard_neg_items[u]) > 0) and (random.random() < hard_prob):
            neg.append(np.random.choice(user_hard_neg_items[u]))
        else:
            # 아니면 Random Negative (안 본 영화)
            while True:
                n = np.random.randint(0, n_items)
                if n not in user_pos_items[u]: # 안 본 것만
                    neg.append(n)
                    break

    return (torch.LongTensor(users).to(device),
            torch.LongTensor(pos).to(device),
            torch.LongTensor(neg).to(device))

# Loss 함수 (BPR)
def bpr_loss(users, pos, neg, u0, p0, n0, reg=1e-4):
    pos_scores = torch.sum(users * pos, dim=1)
    neg_scores = torch.sum(users * neg, dim=1)
    loss = torch.mean(torch.nn.functional.softplus(-(pos_scores - neg_scores)))
    reg_loss = (1/2) * (u0.norm(2).pow(2) + p0.norm(2).pow(2) + n0.norm(2).pow(2)) / float(len(users))
    return loss + reg * reg_loss

# 평가 함수 (NDCG 등) - 기존과 동일
def ndcg_at_k(rank, k):
    if rank is None or rank >= k: return 0.0
    return 1.0 / math.log2(rank + 2)

def evaluate(model, df_eval, k=10):
    model.eval()
    users_final, items_final = model.get_all_embeddings()
    hits, ndcg, precision, recall, total_users = 0, 0, 0, 0, 0

    with torch.no_grad():
        for u_idx, group in df_eval.groupby('user'):
            total_users += 1
            target_items = set(group['item'].values)
            scores = torch.matmul(users_final[u_idx], items_final.t())

            # Train에서 본 아이템(Positive Only)은 마스킹
            if u_idx in user_pos_items:
                scores[list(user_pos_items[u_idx])] = -1e9

            _, topk = torch.topk(scores, k)
            topk = topk.cpu().tolist()

            num_correct = 0
            dcg, idcg = 0.0, 0.0

            for i, item_id in enumerate(topk):
                if item_id in target_items:
                    num_correct += 1
                    dcg += 1.0 / np.log2(i + 2)

            num_targets = len(target_items)
            for i in range(min(num_targets, k)):
                idcg += 1.0 / np.log2(i + 2)

            if num_correct > 0: hits += 1
            precision += num_correct / k
            recall += num_correct / num_targets
            if idcg > 0: ndcg += dcg / idcg

    return {'HitRate': hits/total_users, 'Precision': precision/total_users, 'Recall': recall/total_users, 'NDCG': ndcg/total_users}

1. **lamda1 = 1e-4**

In [16]:
# =========================================================
# 7. 학습 실행 (Hard Negative Sampling 적용)
# =========================================================
dim = 64
layers = 3
batch_size = 1024
epochs = 50
lr = 1e-3
reg_lambda = 1e-4  # 최적화된 파라미터

model = LightGCN(n_users, n_items, dim, layers, Adj_Matrix).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_recall = 0.0
best_epoch = 0
best_model_path = "best_base_lightgcn.pt"

print("=== Training Start (Base + Hard Negative) ===")
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    # 배치 수는 Positive 데이터 기준
    num_batches = len(train_pos_df) // batch_size + 1

    for _ in range(num_batches):
        # [수정] 샘플링 함수 교체 (Hard Negative 포함)
        users, pos, neg = sample_batch_with_hard_neg(batch_size, user_pos_items, user_hard_neg_items, n_items, hard_prob=0.5)

        # 그래프 전파
        users_final, items_final = model.get_all_embeddings()

        u_f = users_final[users]
        i_pos_f = items_final[pos]
        i_neg_f = items_final[neg]

        u_0 = model.user_emb.weight[users]
        i_pos_0 = model.item_emb.weight[pos]
        i_neg_0 = model.item_emb.weight[neg]

        loss = bpr_loss(u_f, i_pos_f, i_neg_f, u_0, i_pos_0, i_neg_0, reg=reg_lambda)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_metrics = evaluate(model, val_df, k=10)

    print(f"[Epoch {epoch:02d}] Loss: {total_loss/num_batches:.4f} | Val Recall: {val_metrics['Recall']:.4f} | NDCG: {val_metrics['NDCG']:.4f}")

    if val_metrics['Recall'] > best_recall:
        best_recall = val_metrics['Recall']
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_path)
        print(f"   >>> Best Model Updated!")

print(f"\n=== Final Test (Best Epoch {best_epoch}) ===")
model.load_state_dict(torch.load(best_model_path))
test_metrics = evaluate(model, test_df, k=10)
print(f"Hit: {test_metrics['HitRate']:.4f}, Prec: {test_metrics['Precision']:.4f}, Recall: {test_metrics['Recall']:.4f}, NDCG: {test_metrics['NDCG']:.4f}")

=== Training Start (Base + Hard Negative) ===
[Epoch 01] Loss: 0.6888 | Val Recall: 0.0796 | NDCG: 0.0404
   >>> Best Model Updated!
[Epoch 02] Loss: 0.6716 | Val Recall: 0.1366 | NDCG: 0.0715
   >>> Best Model Updated!
[Epoch 03] Loss: 0.6121 | Val Recall: 0.1306 | NDCG: 0.0677
[Epoch 04] Loss: 0.5274 | Val Recall: 0.1246 | NDCG: 0.0655
[Epoch 05] Loss: 0.4738 | Val Recall: 0.1246 | NDCG: 0.0661
[Epoch 06] Loss: 0.4431 | Val Recall: 0.1321 | NDCG: 0.0678
[Epoch 07] Loss: 0.4217 | Val Recall: 0.1306 | NDCG: 0.0678
[Epoch 08] Loss: 0.4118 | Val Recall: 0.1321 | NDCG: 0.0684
[Epoch 09] Loss: 0.3939 | Val Recall: 0.1351 | NDCG: 0.0690
[Epoch 10] Loss: 0.3841 | Val Recall: 0.1351 | NDCG: 0.0702
[Epoch 11] Loss: 0.3760 | Val Recall: 0.1366 | NDCG: 0.0713
[Epoch 12] Loss: 0.3596 | Val Recall: 0.1411 | NDCG: 0.0724
   >>> Best Model Updated!
[Epoch 13] Loss: 0.3513 | Val Recall: 0.1381 | NDCG: 0.0721
[Epoch 14] Loss: 0.3374 | Val Recall: 0.1351 | NDCG: 0.0725
[Epoch 15] Loss: 0.3278 | Val Rec

**2. lamda1 = 1e-3**

In [17]:
# =========================================================

dim = 64
layers = 3
batch_size = 1024
epochs = 50
lr = 1e-3
reg_lambda = 1e-3

model = LightGCN(n_users, n_items, dim, layers, Adj_Matrix).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_recall = 0.0
best_epoch = 0
best_model_path = "best_base_lightgcn.pt"

print("=== Training Start (Base + Hard Negative) ===")
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    # 배치 수는 Positive 데이터 기준
    num_batches = len(train_pos_df) // batch_size + 1

    for _ in range(num_batches):
        # [수정] 샘플링 함수 교체 (Hard Negative 포함)
        users, pos, neg = sample_batch_with_hard_neg(batch_size, user_pos_items, user_hard_neg_items, n_items, hard_prob=0.5)

        # 그래프 전파
        users_final, items_final = model.get_all_embeddings()

        u_f = users_final[users]
        i_pos_f = items_final[pos]
        i_neg_f = items_final[neg]

        u_0 = model.user_emb.weight[users]
        i_pos_0 = model.item_emb.weight[pos]
        i_neg_0 = model.item_emb.weight[neg]

        loss = bpr_loss(u_f, i_pos_f, i_neg_f, u_0, i_pos_0, i_neg_0, reg=reg_lambda)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_metrics = evaluate(model, val_df, k=10)

    print(f"[Epoch {epoch:02d}] Loss: {total_loss/num_batches:.4f} | Val Recall: {val_metrics['Recall']:.4f} | NDCG: {val_metrics['NDCG']:.4f}")

    if val_metrics['Recall'] > best_recall:
        best_recall = val_metrics['Recall']
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_path)
        print(f"   >>> Best Model Updated!")

print(f"\n=== Final Test (Best Epoch {best_epoch}) ===")
model.load_state_dict(torch.load(best_model_path))
test_metrics = evaluate(model, test_df, k=10)
print(f"Hit: {test_metrics['HitRate']:.4f}, Prec: {test_metrics['Precision']:.4f}, Recall: {test_metrics['Recall']:.4f}, NDCG: {test_metrics['NDCG']:.4f}")

=== Training Start (Base + Hard Negative) ===
[Epoch 01] Loss: 0.6900 | Val Recall: 0.0781 | NDCG: 0.0402
   >>> Best Model Updated!
[Epoch 02] Loss: 0.6741 | Val Recall: 0.1351 | NDCG: 0.0707
   >>> Best Model Updated!
[Epoch 03] Loss: 0.6151 | Val Recall: 0.1246 | NDCG: 0.0677
[Epoch 04] Loss: 0.5302 | Val Recall: 0.1216 | NDCG: 0.0649
[Epoch 05] Loss: 0.4756 | Val Recall: 0.1246 | NDCG: 0.0657
[Epoch 06] Loss: 0.4459 | Val Recall: 0.1291 | NDCG: 0.0674
[Epoch 07] Loss: 0.4266 | Val Recall: 0.1336 | NDCG: 0.0694
[Epoch 08] Loss: 0.4145 | Val Recall: 0.1366 | NDCG: 0.0712
   >>> Best Model Updated!
[Epoch 09] Loss: 0.3965 | Val Recall: 0.1396 | NDCG: 0.0719
   >>> Best Model Updated!
[Epoch 10] Loss: 0.3898 | Val Recall: 0.1396 | NDCG: 0.0731
[Epoch 11] Loss: 0.3715 | Val Recall: 0.1396 | NDCG: 0.0734
[Epoch 12] Loss: 0.3611 | Val Recall: 0.1456 | NDCG: 0.0750
   >>> Best Model Updated!
[Epoch 13] Loss: 0.3471 | Val Recall: 0.1456 | NDCG: 0.0757
[Epoch 14] Loss: 0.3364 | Val Recall: 0

**3. lamda1 = 1e-2**

In [19]:
# =========================================================

dim = 64
layers = 3
batch_size = 1024
epochs = 25
lr = 1e-3
reg_lambda = 1e-2

model = LightGCN(n_users, n_items, dim, layers, Adj_Matrix).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_recall = 0.0
best_epoch = 0
best_model_path = "best_base_lightgcn.pt"

print("=== Training Start (Base + Hard Negative) ===")
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    # 배치 수는 Positive 데이터 기준
    num_batches = len(train_pos_df) // batch_size + 1

    for _ in range(num_batches):
        # [수정] 샘플링 함수 교체 (Hard Negative 포함)
        users, pos, neg = sample_batch_with_hard_neg(batch_size, user_pos_items, user_hard_neg_items, n_items, hard_prob=0.5)

        # 그래프 전파
        users_final, items_final = model.get_all_embeddings()

        u_f = users_final[users]
        i_pos_f = items_final[pos]
        i_neg_f = items_final[neg]

        u_0 = model.user_emb.weight[users]
        i_pos_0 = model.item_emb.weight[pos]
        i_neg_0 = model.item_emb.weight[neg]

        loss = bpr_loss(u_f, i_pos_f, i_neg_f, u_0, i_pos_0, i_neg_0, reg=reg_lambda)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_metrics = evaluate(model, val_df, k=10)

    print(f"[Epoch {epoch:02d}] Loss: {total_loss/num_batches:.4f} | Val Recall: {val_metrics['Recall']:.4f} | NDCG: {val_metrics['NDCG']:.4f}")

    if val_metrics['Recall'] > best_recall:
        best_recall = val_metrics['Recall']
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_path)
        print(f"   >>> Best Model Updated!")

print(f"\n=== Final Test (Best Epoch {best_epoch}) ===")
model.load_state_dict(torch.load(best_model_path))
test_metrics = evaluate(model, test_df, k=10)
print(f"Hit: {test_metrics['HitRate']:.4f}, Prec: {test_metrics['Precision']:.4f}, Recall: {test_metrics['Recall']:.4f}, NDCG: {test_metrics['NDCG']:.4f}")

=== Training Start (Base + Hard Negative) ===
[Epoch 01] Loss: 0.6980 | Val Recall: 0.0826 | NDCG: 0.0440
   >>> Best Model Updated!
[Epoch 02] Loss: 0.6817 | Val Recall: 0.1231 | NDCG: 0.0641
   >>> Best Model Updated!
[Epoch 03] Loss: 0.6247 | Val Recall: 0.1201 | NDCG: 0.0639
[Epoch 04] Loss: 0.5493 | Val Recall: 0.1201 | NDCG: 0.0643
[Epoch 05] Loss: 0.5005 | Val Recall: 0.1246 | NDCG: 0.0651
   >>> Best Model Updated!
[Epoch 06] Loss: 0.4795 | Val Recall: 0.1261 | NDCG: 0.0661
   >>> Best Model Updated!
[Epoch 07] Loss: 0.4684 | Val Recall: 0.1306 | NDCG: 0.0688
   >>> Best Model Updated!
[Epoch 08] Loss: 0.4585 | Val Recall: 0.1321 | NDCG: 0.0691
   >>> Best Model Updated!
[Epoch 09] Loss: 0.4507 | Val Recall: 0.1321 | NDCG: 0.0696
[Epoch 10] Loss: 0.4398 | Val Recall: 0.1336 | NDCG: 0.0709
   >>> Best Model Updated!
[Epoch 11] Loss: 0.4313 | Val Recall: 0.1381 | NDCG: 0.0731
   >>> Best Model Updated!
[Epoch 12] Loss: 0.4227 | Val Recall: 0.1426 | NDCG: 0.0755
   >>> Best Model 

**4. 1e-5**

In [20]:
# =========================================================

dim = 64
layers = 3
batch_size = 1024
epochs = 25
lr = 1e-3
reg_lambda = 1e-5

model = LightGCN(n_users, n_items, dim, layers, Adj_Matrix).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_recall = 0.0
best_epoch = 0
best_model_path = "best_base_lightgcn.pt"

print("=== Training Start (Base + Hard Negative) ===")
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    # 배치 수는 Positive 데이터 기준
    num_batches = len(train_pos_df) // batch_size + 1

    for _ in range(num_batches):
        # [수정] 샘플링 함수 교체 (Hard Negative 포함)
        users, pos, neg = sample_batch_with_hard_neg(batch_size, user_pos_items, user_hard_neg_items, n_items, hard_prob=0.5)

        # 그래프 전파
        users_final, items_final = model.get_all_embeddings()

        u_f = users_final[users]
        i_pos_f = items_final[pos]
        i_neg_f = items_final[neg]

        u_0 = model.user_emb.weight[users]
        i_pos_0 = model.item_emb.weight[pos]
        i_neg_0 = model.item_emb.weight[neg]

        loss = bpr_loss(u_f, i_pos_f, i_neg_f, u_0, i_pos_0, i_neg_0, reg=reg_lambda)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_metrics = evaluate(model, val_df, k=10)

    print(f"[Epoch {epoch:02d}] Loss: {total_loss/num_batches:.4f} | Val Recall: {val_metrics['Recall']:.4f} | NDCG: {val_metrics['NDCG']:.4f}")

    if val_metrics['Recall'] > best_recall:
        best_recall = val_metrics['Recall']
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_path)
        print(f"   >>> Best Model Updated!")

print(f"\n=== Final Test (Best Epoch {best_epoch}) ===")
model.load_state_dict(torch.load(best_model_path))
test_metrics = evaluate(model, test_df, k=10)
print(f"Hit: {test_metrics['HitRate']:.4f}, Prec: {test_metrics['Precision']:.4f}, Recall: {test_metrics['Recall']:.4f}, NDCG: {test_metrics['NDCG']:.4f}")

=== Training Start (Base + Hard Negative) ===
[Epoch 01] Loss: 0.6890 | Val Recall: 0.0631 | NDCG: 0.0320
   >>> Best Model Updated!
[Epoch 02] Loss: 0.6728 | Val Recall: 0.1336 | NDCG: 0.0692
   >>> Best Model Updated!
[Epoch 03] Loss: 0.6128 | Val Recall: 0.1276 | NDCG: 0.0662
[Epoch 04] Loss: 0.5260 | Val Recall: 0.1261 | NDCG: 0.0662
[Epoch 05] Loss: 0.4681 | Val Recall: 0.1246 | NDCG: 0.0656
[Epoch 06] Loss: 0.4391 | Val Recall: 0.1276 | NDCG: 0.0666
[Epoch 07] Loss: 0.4191 | Val Recall: 0.1351 | NDCG: 0.0694
   >>> Best Model Updated!
[Epoch 08] Loss: 0.4056 | Val Recall: 0.1336 | NDCG: 0.0702
[Epoch 09] Loss: 0.3939 | Val Recall: 0.1396 | NDCG: 0.0718
   >>> Best Model Updated!
[Epoch 10] Loss: 0.3777 | Val Recall: 0.1366 | NDCG: 0.0717
[Epoch 11] Loss: 0.3683 | Val Recall: 0.1396 | NDCG: 0.0727
[Epoch 12] Loss: 0.3515 | Val Recall: 0.1441 | NDCG: 0.0743
   >>> Best Model Updated!
[Epoch 13] Loss: 0.3418 | Val Recall: 0.1426 | NDCG: 0.0747
[Epoch 14] Loss: 0.3303 | Val Recall: 0

In [21]:
# =========================================================

dim = 64
layers = 3
batch_size = 1024
epochs = 25
lr = 1e-3
reg_lambda = 1e-6

model = LightGCN(n_users, n_items, dim, layers, Adj_Matrix).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_recall = 0.0
best_epoch = 0
best_model_path = "best_base_lightgcn.pt"

print("=== Training Start (Base + Hard Negative) ===")
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    # 배치 수는 Positive 데이터 기준
    num_batches = len(train_pos_df) // batch_size + 1

    for _ in range(num_batches):
        # [수정] 샘플링 함수 교체 (Hard Negative 포함)
        users, pos, neg = sample_batch_with_hard_neg(batch_size, user_pos_items, user_hard_neg_items, n_items, hard_prob=0.5)

        # 그래프 전파
        users_final, items_final = model.get_all_embeddings()

        u_f = users_final[users]
        i_pos_f = items_final[pos]
        i_neg_f = items_final[neg]

        u_0 = model.user_emb.weight[users]
        i_pos_0 = model.item_emb.weight[pos]
        i_neg_0 = model.item_emb.weight[neg]

        loss = bpr_loss(u_f, i_pos_f, i_neg_f, u_0, i_pos_0, i_neg_0, reg=reg_lambda)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_metrics = evaluate(model, val_df, k=10)

    print(f"[Epoch {epoch:02d}] Loss: {total_loss/num_batches:.4f} | Val Recall: {val_metrics['Recall']:.4f} | NDCG: {val_metrics['NDCG']:.4f}")

    if val_metrics['Recall'] > best_recall:
        best_recall = val_metrics['Recall']
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_path)
        print(f"   >>> Best Model Updated!")

print(f"\n=== Final Test (Best Epoch {best_epoch}) ===")
model.load_state_dict(torch.load(best_model_path))
test_metrics = evaluate(model, test_df, k=10)
print(f"Hit: {test_metrics['HitRate']:.4f}, Prec: {test_metrics['Precision']:.4f}, Recall: {test_metrics['Recall']:.4f}, NDCG: {test_metrics['NDCG']:.4f}")

=== Training Start (Base + Hard Negative) ===
[Epoch 01] Loss: 0.6887 | Val Recall: 0.0841 | NDCG: 0.0473
   >>> Best Model Updated!
[Epoch 02] Loss: 0.6707 | Val Recall: 0.1396 | NDCG: 0.0733
   >>> Best Model Updated!
[Epoch 03] Loss: 0.6089 | Val Recall: 0.1276 | NDCG: 0.0685
[Epoch 04] Loss: 0.5228 | Val Recall: 0.1231 | NDCG: 0.0661
[Epoch 05] Loss: 0.4713 | Val Recall: 0.1261 | NDCG: 0.0660
[Epoch 06] Loss: 0.4387 | Val Recall: 0.1291 | NDCG: 0.0675
[Epoch 07] Loss: 0.4180 | Val Recall: 0.1336 | NDCG: 0.0696
[Epoch 08] Loss: 0.4059 | Val Recall: 0.1366 | NDCG: 0.0709
[Epoch 09] Loss: 0.3906 | Val Recall: 0.1381 | NDCG: 0.0716
[Epoch 10] Loss: 0.3805 | Val Recall: 0.1381 | NDCG: 0.0725
[Epoch 11] Loss: 0.3666 | Val Recall: 0.1411 | NDCG: 0.0741
   >>> Best Model Updated!
[Epoch 12] Loss: 0.3566 | Val Recall: 0.1381 | NDCG: 0.0739
[Epoch 13] Loss: 0.3442 | Val Recall: 0.1411 | NDCG: 0.0746
[Epoch 14] Loss: 0.3310 | Val Recall: 0.1396 | NDCG: 0.0747
[Epoch 15] Loss: 0.3198 | Val Rec